In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv("phosphide_mo.csv")

In [20]:
df.head()

,catalyst,catalyst_family,metal_component,contains_noble_metal,pH,temperature_C,tafel_slope_mV_dec,exchange_current_density_A_cm2,roughness_factor,ECSA_cm2,is_composite,morphology_quality_score,overpotential_mV_10mAcm2,_sheet_name,__synthetic__
0,MoPh-mesh-53,Phosphide,Mo,0,5.129075,47,119.428204,0.390072,0.760334,1.156723,0,0.527570,98.291129,sheet1,True
1,MoPh-thin-32,Phosphide,Mo,0,12.435733,84,198.088620,0.368063,1.761763,8.254903,0,0.571813,154.875181,sheet1,True
2,MoPh-nano-15,Phosphide,Mo,0,8.222794,45,63.656148,0.040783,2.675036,3.533272,1,0.554740,60.664145,sheet1,True
3,MoPh-mesh-53,Phosphide,Mo,0,4.729383,47,123.049486,0.400182,0.759221,1.240783,0,0.519377,97.321501,sheet1,True
4,MoPh-nano-19,Phosphide,Mo,0,8.615098,32,64.670567,0.684649,2.263545,4.537731,1,0.988345,177.422692,sheet1,True


In [21]:
df = df.drop(columns=['roughness_factor','morphology_quality_score','_sheet_name','__synthetic__','contains_noble_metal','catalyst'])

In [22]:
df.head()

,catalyst_family,metal_component,pH,temperature_C,tafel_slope_mV_dec,exchange_current_density_A_cm2,ECSA_cm2,is_composite,overpotential_mV_10mAcm2
0,Phosphide,Mo,5.129075,47,119.428204,0.390072,1.156723,0,98.291129
1,Phosphide,Mo,12.435733,84,198.088620,0.368063,8.254903,0,154.875181
2,Phosphide,Mo,8.222794,45,63.656148,0.040783,3.533272,1,60.664145
3,Phosphide,Mo,4.729383,47,123.049486,0.400182,1.240783,0,97.321501
4,Phosphide,Mo,8.615098,32,64.670567,0.684649,4.537731,1,177.422692


In [23]:
bins = [0, 150, 250, df["overpotential_mV_10mAcm2"].max() + 1] # Add +1 to max for safety
labels = ["Low", "Medium", "High"]
df["overpotential_class"] = pd.cut(df["overpotential_mV_10mAcm2"], bins=bins, labels=labels, right=False) # Use right=False for (0, 150] etc.


bins_tafel = [0, 80, 150, df["tafel_slope_mV_dec"].max() + 1]
df["tafel_class"] = pd.cut(df["tafel_slope_mV_dec"], bins=bins_tafel, labels=labels, right=False)


df['performance_class'] = 1 # Default is Poor
df.loc[(df['overpotential_class'] == 'Low') & (df['tafel_class'] == 'Low'), 'performance_class'] = 3
df.loc[((df['overpotential_class'] == 'Low') & (df['tafel_class'] == 'Medium')) | ((df['overpotential_class'] == 'Medium') & (df['tafel_class'] == 'Low')), 'performance_class'] = 2
df.loc[((df['overpotential_class'] == 'Medium') & (df['tafel_class'] == 'Medium')) | ((df['overpotential_class'] == 'High') & (df['tafel_class'] == 'Low')) | ((df['overpotential_class'] == 'Low') & (df['tafel_class'] == 'High')),'performance_class'] = 1



In [24]:
df.sample(10)

,catalyst_family,metal_component,pH,temperature_C,tafel_slope_mV_dec,exchange_current_density_A_cm2,ECSA_cm2,is_composite,overpotential_mV_10mAcm2,overpotential_class,tafel_class,performance_class
282,Phosphide,Mo,13.990072,48,45.769511,0.189312,3.718386,0,97.371207,Low,Low,3
302,Phosphide,Mo,1.630753,66,171.812780,0.694455,3.495646,1,179.383125,Medium,High,1
256,Phosphide,Mo,5.024175,45,124.575940,0.405518,1.089370,0,100.659185,Low,Medium,2
22,Phosphide,Mo,6.836992,43,81.574029,0.826747,4.261213,1,224.531062,Medium,Medium,1
59,Phosphide,Mo,7.887641,42,61.399020,0.018372,3.868709,1,57.962635,Low,Low,3
335,Phosphide,Mo,8.389248,31,70.515732,0.655122,4.765327,1,183.297228,Medium,Low,2
154,Phosphide,Mo,12.222135,85,195.924599,0.367539,8.180756,0,153.910397,Medium,High,1
291,Phosphide,Mo,8.651205,30,65.981715,0.656772,4.570759,1,179.034534,Medium,Low,2
70,Phosphide,Mo,1.702331,64,166.581151,0.731632,3.776057,1,176.132126,Medium,High,1
72,Phosphide,Mo,13.601839,47,37.848989,0.196927,3.497145,0,93.081678,Low,Low,3


In [25]:
df.columns

Index(['catalyst_family', 'metal_component', 'pH', 'temperature_C',
       'tafel_slope_mV_dec', 'exchange_current_density_A_cm2', 'ECSA_cm2',
       'is_composite', 'overpotential_mV_10mAcm2', 'overpotential_class',
       'tafel_class', 'performance_class'],
      dtype='object')

In [26]:
df.to_csv('temp_pH_data.csv',index=False)

In [27]:
data = pd.read_csv("modified_data.csv")

In [28]:
import pandas as pd
import plotly.express as px

data['performance_class'] = data['performance_class'].astype(str)

class_order = ['3', '2', '1']
color_map = {
    '3': 'lightgreen',    # Best performance
    '2': 'pink',
    '1': 'lavender'       # Worst performance
}


fig = px.scatter(
    data,
    x='pH',
    y='temperature_C',
    color='performance_class',
    category_orders={'performance_class': class_order}, 
    color_discrete_map=color_map, 
    hover_data=data.columns.tolist(), 
    title='Catalyst Performance by Working pH and Temperature',
    labels={
        'pH': 'Working pH',
        'temperature_C': 'Temperature (°C)',
        'performance_class': 'Performance Class (3=Best)',
        'overpotential_mV_10mAcm2': 'overpotential_mV_10mAcm2',
        'tafel_slope_mV_dec' : 'tafel_slope_mV_dec'
    }
)

fig.update_traces(
    marker=dict(
        sizemin=5, 
        line=dict(width=1, color='DarkSlateGrey')
    ),
    selector=dict(mode='markers')
)

fig.update_layout(
    xaxis_title='Working pH',
    yaxis_title='Temperature (°C)',
    legend_title='Performance Class',
    hovermode='closest'
)

fig.show("iframe")
